In [1]:
import sys
print(sys.executable)
import strax
import straxen
print('Straxen: ', straxen.__version__, straxen.__file__)
print('Strax: ', strax.__version__, strax.__file__)
import configparser as cp

sys.path.append('/home/gvolta/XENONnT/gain_spe')
from PMTanalysis import *
PMT = PMTanalyzer('/home/gvolta/XENONnT/gain_spe')

/dali/lgrandi/strax/miniconda3/envs/strax/bin/python
Straxen:  0.7.3 /home/gvolta/XENONnT/straxen/straxen/__init__.py
Strax:  0.10.3 /dali/lgrandi/strax/strax/strax/__init__.py




___   ___  _______ .__   __.   ______   .__   __.        .___________.   __  __             _____    _____    ____  
\  \ /  / |   ____||  \ |  |  /  __  \  |  \ |  |        |           |  |  \/  |    /\     / ____|  |_   _| /  ___| 
 \  V  /  |  |__   |   \|  | |  |  |  | |   \|  |  _   _ `---|  |----`  | \  / |   /  \    | |  __    | |   | |      
  >   <   |   __|  |  . `  | |  |  |  | |  . `  | | \ | |    |  |       | |\/| |  / /\ \   | | |_ |   | |   | |      
 /  .  \  |  |____ |  |\   | |  `--`  | |  |\   | |  \| |    |  |       | |  | | / ____ \  | |__| |  _| |_  | |___  
/__/ \__\ |_______||__| \__|  \______/  |__| \__| |_|\__|    |__|       |_|  |_|/_/    \_\ \ _____| |_____| \_____|  
powered by Giovanni Volta and Chiara Capelli (c)




Configuration file:  /home/gvolta/XENONnT/gain_spe/configurat

In [2]:
strax.Mailbox.DEFAULT_MAX_MESSAGES = 2
st = straxen.contexts.xenonnt_online()

In [16]:
run = st.select_runs(run_id='7212')
run.

,number,mode,start,end,name,tags,livetime,reader.ini.name,trigger.events_built,strax_defaults,tags.name,raw_records_available
7212,7212,,2020-04-15 15:38:19.055,2020-04-15 15:38:49.345,7212,,00:00:30.290000,NaN,NaN,NaN,NaN,False


In [9]:
!ls /dali/lgrandi/xenonnt/raw/

007154-raw_records_aqmon-rfzvpzj4mf  007193-raw_records_mv-rfzvpzj4mf
007154-raw_records-rfzvpzj4mf	     007193-raw_records-rfzvpzj4mf
007157-raw_records-rfzvpzj4mf	     007194-raw_records_aqmon-rfzvpzj4mf
007158-raw_records_aqmon-rfzvpzj4mf  007194-raw_records_he-rfzvpzj4mf
007158-raw_records_he-rfzvpzj4mf     007194-raw_records_mv-rfzvpzj4mf
007158-raw_records-rfzvpzj4mf	     007194-raw_records-rfzvpzj4mf
007159-raw_records_aqmon-rfzvpzj4mf  007196-raw_records_aqmon-rfzvpzj4mf
007159-raw_records_he-rfzvpzj4mf     007196-raw_records_he-rfzvpzj4mf
007159-raw_records-rfzvpzj4mf	     007196-raw_records_mv-rfzvpzj4mf
007177-raw_records_aqmon-rfzvpzj4mf  007196-raw_records-rfzvpzj4mf
007177-raw_records_he-rfzvpzj4mf     007197-raw_records_aqmon-rfzvpzj4mf
007177-raw_records_mv-rfzvpzj4mf     007197-raw_records_he-rfzvpzj4mf
007177-raw_records-rfzvpzj4mf	     007197-raw_records_mv-rfzvpzj4mf
007178-raw_records_aqmon-rfzvpzj4mf  007197-raw_records-rfzvpzj4mf
007178-raw_records_he-rfzvpzj4mf 

In [19]:
data_rr = st.get_array('7212', 'raw_records')

DataNotAvailable: raw_records for 7212 not found in any storage, and your context specifies it cannot be created.